<a href="https://colab.research.google.com/github/wiiiiish/2022-machine_learning/blob/main/%EC%98%88%EC%B8%A1%EA%B0%92_DB_%EC%A0%80%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!ls

drive  sample_data


In [6]:
%cd drive/MyDrive/Colab\ Notebooks/2022-04-27\ 데이터/2022-05-13\ 모델\ 생성\ 작업

/content/drive/MyDrive/Colab Notebooks/2022-04-27 데이터/2022-05-13 모델 생성 작업


In [7]:
!sudo apt-get install python3-dev default-libmysqlclient-dev
!pip install pymysql

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-libmysqlclient-dev is already the newest version (1.0.4).
default-libmysqlclient-dev set to manually installed.
python3-dev is already the newest version (3.6.7-1~18.04).
python3-dev set to manually installed.
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.
     |████████████████████████████████| 43 kB 2.0 MB/s 


In [8]:
import pandas as pd
import numpy as np
import pymysql # mysql 연결
import pickle # 모델 불러올 때 사용
from datetime import date, datetime, timezone, timedelta # 시간 관련 라이브러리
import pickle

from pymysql.converters import convert_bit

In [10]:
# 시간 맞추기
# hours=9 는 현재 한국시간
# 우리는 한 시간 후의 시간을 불러와야 하기 때문에 hours=10 으로 설정
K_KST = timezone(timedelta(hours=9))
A_KST = timezone(timedelta(hours=10))
B_KST = timezone(timedelta(hours=8))
T_KST = timezone(timedelta(hours=7))


In [9]:
from keras.models import load_model
model_rnn = load_model('./rnn_model_v2.h5')

In [ ]:
K_KST = timezone(timedelta(hours=9))
A_KST = timezone(timedelta(hours=10))
B_KST = timezone(timedelta(hours=8))
T_KST = timezone(timedelta(hours=7))

while True:
  try :
    # 현재 시간 구하기
    now = datetime.now(K_KST)
    current_time = now.strftime('%Y-%m-%d %H:%M:%S')
    s_current_time = f"{current_time}"
    s_current_time = s_current_time[:18]
    print(f"현재 시간 : {s_current_time}") # 현재 시간 / 11:11:12 -> 11:11:1 로 자르기

    # 한 시간 전, 후의 시간
    after = datetime.now(A_KST)
    after_time = after.strftime('%Y-%m-%d %H:%M:%S')
    s_after_time = f"{after_time}"
    s_after_time = s_after_time[:18]
    print(f"한 시간 후 : {s_after_time}")

    before = datetime.now(B_KST)
    before_time = before.strftime('%Y-%m-%d %H:%M:%S')
    s_before_time = f"{before_time}"
    s_before_time = s_before_time[:18]
    print(f"한 시간 전 : {s_before_time}")

    # 두 시간 전의 시간
    before_two = datetime.now(T_KST)
    before_two_time = before_two.strftime('%Y-%m-%d %H:%M:%S')
    s_before_two_time = f"{before_two_time}"
    s_before_two_time = s_before_two_time[:18]
    print(f"두 시간 전 : {s_before_two_time}")

    # DB 연결
    from pymysql.converters import convert_bit
    host = "project-db-stu.ddns.net"
    port = 3307
    username = "wish"
    password = "1234"
    db = 'wish'

    conn = pymysql.connect(host=host, port=port, user=username, password=password, db=db)

    # select 현재 시간의 데이터 번호
    sql_state1 = f"select r_seq from s_real_data where r_date like '{s_current_time}0'"
    s_real = pd.read_sql_query(sql_state1, conn)
    s_r_seq = s_real['r_seq']

    # 한 시간 후의 날씨 데이터 목록
    sql_state2 = f"select r_temperature, r_precipitation, r_wind, r_humidity, r_radiation, r_total_cloudiness, r_visibility, r_ground_temp from s_real_data where r_date > '{s_current_time}0' and r_date <= '{s_after_time}0' order by r_date asc"
    s_after_weather = pd.read_sql_query(sql_state2, conn)
    after_hour_weather = pd.DataFrame(s_after_weather)

    # select 현재 발전량 목록
    sql_state3 = f"select r_aod from s_real_data where r_date > '{s_before_time}0' and r_date <= '{s_current_time}0' order by r_date asc"
    s_now_aod = pd.read_sql_query(sql_state3, conn)
    now_aod = pd.DataFrame(s_now_aod)

    # select 한 시간 전 발전량 목록
    sql_state4 = f"select r_aod from s_real_data where r_date > '{s_before_two_time}0' and r_date <= '{s_before_time}0'"
    s_before_aod = pd.read_sql_query(sql_state4, conn)
    before_aod = pd.DataFrame(s_before_aod)

    # 학습데이터
    data = pd.concat([after_hour_weather, now_aod, before_aod], axis=1)
    print(data)

    s_data = np.array(data)

    s_s_data = s_data.reshape(1,360,10)
    pred = model_rnn.predict(s_s_data)

    s_pred = pred[0][0]
    print(s_pred)
    display(s_r_seq)

    # DB insert into s_predict
    sql_state5 = f"insert into s_prediction values (null, {int(s_r_seq[0])}, '{s_after_time}0', {s_pred})"
    cursor = conn.cursor()
    cursor.execute(sql_state5)
    conn.commit()

  except:
    print("예외 발생!")

현재 시간 : 2022-05-17 09:53:4
한 시간 후 : 2022-05-17 10:53:4
한 시간 전 : 2022-05-17 08:53:4
두 시간 전 : 2022-05-17 07:53:4
     r_temperature  r_precipitation  r_wind  r_humidity  r_radiation  \
0             23.2              0.0     0.9        74.1          1.0   
1             23.3              0.0     0.8        79.3          1.0   
2             25.0              0.0     0.6        82.1          0.9   
3             23.9              0.0     0.8        86.0          0.9   
4             24.4              0.0     0.7        75.8          1.0   
..             ...              ...     ...         ...          ...   
355           24.8              0.0     0.8        72.1          1.0   
356           24.9              0.0     0.8        73.7          0.9   
357           24.8              0.0     0.9        72.1          1.0   
358           24.8              0.0     0.8        70.2          0.9   
359           25.0              0.0     0.9        74.4          0.9   

     r_total_cloudiness 

0    20843
Name: r_seq, dtype: int64

현재 시간 : 2022-05-17 09:53:4
한 시간 후 : 2022-05-17 10:53:4
한 시간 전 : 2022-05-17 08:53:4
두 시간 전 : 2022-05-17 07:53:4
     r_temperature  r_precipitation  r_wind  r_humidity  r_radiation  \
0             23.2              0.0     0.9        74.1          1.0   
1             23.3              0.0     0.8        79.3          1.0   
2             25.0              0.0     0.6        82.1          0.9   
3             23.9              0.0     0.8        86.0          0.9   
4             24.4              0.0     0.7        75.8          1.0   
..             ...              ...     ...         ...          ...   
355           24.8              0.0     0.8        72.1          1.0   
356           24.9              0.0     0.8        73.7          0.9   
357           24.8              0.0     0.9        72.1          1.0   
358           24.8              0.0     0.8        70.2          0.9   
359           25.0              0.0     0.9        74.4          0.9   

     r_total_cloudiness 

0    20843
Name: r_seq, dtype: int64

현재 시간 : 2022-05-17 09:53:4
한 시간 후 : 2022-05-17 10:53:4
한 시간 전 : 2022-05-17 08:53:4
두 시간 전 : 2022-05-17 07:53:4
     r_temperature  r_precipitation  r_wind  r_humidity  r_radiation  \
0             23.2              0.0     0.9        74.1          1.0   
1             23.3              0.0     0.8        79.3          1.0   
2             25.0              0.0     0.6        82.1          0.9   
3             23.9              0.0     0.8        86.0          0.9   
4             24.4              0.0     0.7        75.8          1.0   
..             ...              ...     ...         ...          ...   
355           24.8              0.0     0.8        72.1          1.0   
356           24.9              0.0     0.8        73.7          0.9   
357           24.8              0.0     0.9        72.1          1.0   
358           24.8              0.0     0.8        70.2          0.9   
359           25.0              0.0     0.9        74.4          0.9   

     r_total_cloudiness 

...

현재 시간 : 2022-05-17 09:54:0
한 시간 후 : 2022-05-17 10:54:0
한 시간 전 : 2022-05-17 08:54:0
두 시간 전 : 2022-05-17 07:54:0
     r_temperature  r_precipitation  r_wind  r_humidity  r_radiation  \
0             25.0              0.0     0.6        82.1          0.9   
1             23.9              0.0     0.8        86.0          0.9   
2             24.4              0.0     0.7        75.8          1.0   
3             24.3              0.0     0.7        83.3          1.0   
4             23.9              0.0     0.6        75.8          1.0   
..             ...              ...     ...         ...          ...   
355           24.8              0.0     0.9        72.1          1.0   
356           24.8              0.0     0.8        70.2          0.9   
357           25.0              0.0     0.9        74.4          0.9   
358           24.9              0.0     0.8        72.6          1.1   
359           24.9              0.0     0.9        72.4          1.0   

     r_total_cloudiness 

0    20845
Name: r_seq, dtype: int64

현재 시간 : 2022-05-17 09:54:0
한 시간 후 : 2022-05-17 10:54:0
한 시간 전 : 2022-05-17 08:54:0
두 시간 전 : 2022-05-17 07:54:0
     r_temperature  r_precipitation  r_wind  r_humidity  r_radiation  \
0             25.0              0.0     0.6        82.1          0.9   
1             23.9              0.0     0.8        86.0          0.9   
2             24.4              0.0     0.7        75.8          1.0   
3             24.3              0.0     0.7        83.3          1.0   
4             23.9              0.0     0.6        75.8          1.0   
..             ...              ...     ...         ...          ...   
355           24.8              0.0     0.9        72.1          1.0   
356           24.8              0.0     0.8        70.2          0.9   
357           25.0              0.0     0.9        74.4          0.9   
358           24.9              0.0     0.8        72.6          1.1   
359           24.9              0.0     0.9        72.4          1.0   

     r_total_cloudiness 

0    20845
Name: r_seq, dtype: int64

현재 시간 : 2022-05-17 09:54:0
한 시간 후 : 2022-05-17 10:54:0
한 시간 전 : 2022-05-17 08:54:0
두 시간 전 : 2022-05-17 07:54:0


In [11]:
# DB 연결

host = "project-db-stu.ddns.net"
port = 3307
username = "wish"
password = "1234"
db = 'wish'

conn = pymysql.connect(host=host, port=port, user=username, password=password, db=db)

# select 현재 시간, 데이터 번호 (real_data의 마지막 데이터 불러오기)
sql_state1 = f"select r_seq, r_date, r_aod from s_real_data order by r_date desc limit 1"
s_real = pd.read_sql_query(sql_state1, conn)
s_r_seq = s_real['r_seq']
s_r_date = s_real['r_date']
s_r_now_aod = s_real['r_aod']

# 불러온 데이터의 시간 추출
s_now_time = f"{s_r_date[0]}"
print(f"s_now_time : {s_now_time}")

# s_now_time 의 한 시간 후의 시간
s_now_time_slice = s_now_time[:18]
s_pred_t_hour = (int)(s_now_time_slice[11:13]) + 1
print(f"s_pred_t_hour : {s_pred_t_hour}")

if s_pred_t_hour < 10:
  s_pred_t_hour = f"0{s_pred_t_hour}"
elif s_pred_t_hour > 23:
  s_pred_t_hour = "00"

s_pred_t_sec = (int)(s_now_time_slice[17]) + 1

if s_pred_t_sec == 6:
  s_pred_t_sec = "0"

print(f"s_pred_t_sec : {s_pred_t_sec}")

timeList = list(s_now_time)
timeList[11:13] = f"{s_pred_t_hour}"
timeList[17] = f"{s_pred_t_sec}"
s_pred_time = ''.join(s for s in timeList)
s_pred_time_slice = s_pred_time[:18]
print(f"s_pred_time : {s_pred_time}")
print(f"s_pred_time_slice : {s_pred_time_slice}")

# s_now_time 의 한 시간 전의 시간
s_before_t = (int)(s_now_time_slice[11:13]) - 1
print(f"s_before_t : {s_before_t}")

if s_before_t < 0:
  s_before_t = "23"
elif s_before_t < 10:
  s_before_t = f"0{s_before_t}"
    
timeList[11:13] = f"{s_before_t}"
s_before_time = ''.join(s for s in timeList)
s_before_time_slice = s_before_time[:18]
print(f"s_before_time_slice : {s_before_time_slice}")

s_now_time : 2022-05-16 11:36:50
s_pred_t_hour : 12
s_pred_t_sec : 0
s_pred_time : 2022-05-16 12:36:00
s_pred_time_slice : 2022-05-16 12:36:0
s_before_t : 10
s_before_time_slice : 2022-05-16 10:36:0


In [15]:
K_KST = timezone(timedelta(hours=9))
A_KST = timezone(timedelta(hours=10))
B_KST = timezone(timedelta(hours=8))
T_KST = timezone(timedelta(hours=7))

# 현재 시간 구하기
now = datetime.now(K_KST)
current_time = now.strftime('%Y-%m-%d %H:%M:%S')
s_current_time = f"{current_time}"
s_current_time = s_current_time[:18]
print(f"현재 시간 : {s_current_time}") # 현재 시간 / 11:11:12 -> 11:11:1 로 자르기

# 한 시간 전, 후의 시간
after = datetime.now(A_KST)
after_time = after.strftime('%Y-%m-%d %H:%M:%S')
s_after_time = f"{after_time}"
s_after_time = s_after_time[:18]
print(f"한 시간 후 : {s_after_time}")

before = datetime.now(B_KST)
before_time = before.strftime('%Y-%m-%d %H:%M:%S')
s_before_time = f"{before_time}"
s_before_time = s_before_time[:18]
print(f"한 시간 전 : {s_before_time}")

# 두 시간 전의 시간
before_two = datetime.now(T_KST)
before_two_time = before_two.strftime('%Y-%m-%d %H:%M:%S')
s_before_two_time = f"{before_two_time}"
s_before_two_time = s_before_two_time[:18]
print(f"두 시간 전 : {s_before_two_time}")

# DB 연결
from pymysql.converters import convert_bit
host = "project-db-stu.ddns.net"
port = 3307
username = "wish"
password = "1234"
db = 'wish'

conn = pymysql.connect(host=host, port=port, user=username, password=password, db=db)

# select 현재 시간의 데이터 번호
sql_state1 = f"select r_seq from s_real_data where r_date like '{s_current_time}0'"
s_real = pd.read_sql_query(sql_state1, conn)
s_r_seq = s_real['r_seq']

# 한 시간 후의 날씨 데이터 목록
sql_state2 = f"select r_temperature, r_precipitation, r_wind, r_humidity, r_radiation, r_total_cloudiness, r_visibility, r_ground_temp from s_real_data where r_date > '{s_current_time}0' and r_date <= '{s_after_time}0' order by r_date asc"
s_after_weather = pd.read_sql_query(sql_state2, conn)
after_hour_weather = pd.DataFrame(s_after_weather)

# select 현재 발전량 목록
sql_state3 = f"select r_aod from s_real_data where r_date > '{s_before_time}0' and r_date <= '{s_current_time}0' order by r_date asc"
s_now_aod = pd.read_sql_query(sql_state3, conn)
now_aod = pd.DataFrame(s_now_aod)

# select 한 시간 전 발전량 목록
sql_state4 = f"select r_aod from s_real_data where r_date > '{s_before_two_time}0' and r_date <= '{s_before_time}0'"
s_before_aod = pd.read_sql_query(sql_state4, conn)
before_aod = pd.DataFrame(s_before_aod)

# 학습데이터
data = pd.concat([after_hour_weather, now_aod, before_aod], axis=1)
print(data)

s_data = np.array(data)

s_s_data = s_data.reshape(1,360,10)
pred = model_rnn.predict(s_s_data)

s_pred = pred[0][0]
print(s_pred)
display(s_r_seq)

# DB insert into s_predict
sql_state5 = f"insert into s_prediction values (null, {int(s_r_seq[0])}, '{s_after_time}0', {s_pred})"
cursor = conn.cursor()
cursor.execute(sql_state5)
conn.commit()



현재 시간 : 2022-05-17 09:52:1
한 시간 후 : 2022-05-17 10:52:1
한 시간 전 : 2022-05-17 08:52:1
두 시간 전 : 2022-05-17 07:52:1
     r_temperature  r_precipitation  r_wind  r_humidity  r_radiation  \
0             23.8              0.0     0.6        80.6          0.9   
1             24.3              0.0     0.7        85.0          1.0   
2             23.2              0.0     0.6        72.3          1.0   
3             23.7              0.0     0.8        80.6          0.9   
4             23.6              0.0     0.6        81.8          0.9   
..             ...              ...     ...         ...          ...   
355           25.0              0.0     0.8        72.3          1.1   
356           25.0              0.0     0.9        73.2          0.9   
357           24.8              0.0     0.8        69.6          1.0   
358           24.9              0.0     0.8        69.9          1.0   
359           24.9              0.0     0.9        72.8          1.0   

     r_total_cloudiness 

0    20834
Name: r_seq, dtype: int64